In [1]:
import json
import csv

In [2]:
"""
This can probably be improved but serves the purpose of our analysis.
I used Chris's schema so it will be useful for Quarz analysis.

"""

EMOTIONS = [
    'contempt',
    'fear',
    'sadness',
    'disgust',
    'anger',
    'neutral',
    'happiness',
    'surprise',
]


CSV_COLUMNS = [
    'url',
    'frame_time',
    'trump_visible',
    'trump_happiness',
    'trump_neutral',
    'trump_anger',
    'trump_contempt',
    'trump_suprise',
    'trump_sadness',
    'trump_fear',
    'trump_disgust',
    'clinton_visible',
    'clinton_happiness',
    'clinton_neutral',
    'clinton_anger',
    'clinton_contempt',
    'clinton_suprise',
    'clinton_sadness',
    'clinton_fear',
    'clinton_disgust'
]

In [3]:
with open('/Users/Amiros/GitHub/debate/debate3_full.json') as data_file:    
    json_data = json.load(data_file)
    
json_data = {int(k):v for k,v in json_data.items()} #converting keys to integer   
frame_keys = sorted(json_data.keys())

In [4]:
csv_data = []
   
for key in frame_keys:
    url = 'https://s3-us-west-2.amazonaws.com/debateinemotion/debate3/3rd_debate%d.jpg' % key
    frame = json_data[key]
        
    row = {
        #'frame': frame,
        'url': url,
        'frame_time': key,
        'trump_visible': False,
        'clinton_visible': False
    }
    
    if len(frame) > 2:
        print('Frame %s has more than 2 faces (%s)' % (key, url))
        continue
    elif len(frame) < 2:
        for face in frame:
            if face['faceAttributes']['gender'] == 'male':
                prefix = 'trump_'
                row[prefix + 'visible'] = True
                for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                    row[prefix + emotion] = face['scores'][emotion]
                ### assign emotions vale to each candidate and change the name to the respective one 
                    
            if face['faceAttributes']['gender'] == 'female':
                prefix = 'clinton_'
                row[prefix + 'visible'] = True
                for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                    row[prefix + emotion] = face['scores'][emotion]
            row.pop("frame", None)        
      
    elif len(frame) == 2:
        row['trump_visible'] = True
        row['clinton_visible'] = True
        for face in frame:
            if face['faceAttributes']['gender'] == 'male':
                prefix = 'trump_'
                row[prefix + 'visible'] = True
                for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                    row[prefix + emotion] = face['scores'][emotion]
                ### assign emotions vale to each candidate and change the name to the respective one 
                    
            if face['faceAttributes']['gender'] == 'female':
                prefix = 'clinton_'
                row[prefix + 'visible'] = True
                for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                    row[prefix + emotion] = face['scores'][emotion]
        row.pop("frame", None)        

    csv_data.append(row)

In [5]:
import pandas as pd
df = pd.DataFrame(csv_data)

In [6]:
df

,clinton_anger,clinton_contempt,clinton_disgust,clinton_fear,clinton_happiness,clinton_neutral,clinton_sadness,clinton_surprise,clinton_visible,frame_time,trump_anger,trump_contempt,trump_disgust,trump_fear,trump_happiness,trump_neutral,trump_sadness,trump_surprise,trump_visible,url
0,0.000522,0.001140,0.002116,4.809360e-05,0.530032,0.255370,0.000190,0.210581,True,355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,https://s3-us-west-2.amazonaws.com/debateinemo...
1,0.000086,0.000274,0.000177,1.324917e-05,0.000497,0.972558,0.001012,0.025382,True,360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,https://s3-us-west-2.amazonaws.com/debateinemo...
2,0.000301,0.010076,0.000263,2.447346e-05,0.011186,0.920105,0.001091,0.056953,True,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,https://s3-us-west-2.amazonaws.com/debateinemo...
3,0.000050,0.004093,0.000296,4.457917e-06,0.004277,0.957873,0.001874,0.031532,True,370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,https://s3-us-west-2.amazonaws.com/debateinemo...
4,0.000656,0.000997,0.001257,2.931259e-05,0.022650,0.756939,0.001206,0.216267,True,375,0.111283,0.013174,0.001539,3.399206e-06,2.607155e-07,0.268892,0.605103,5.915630e-06,True,https://s3-us-west-2.amazonaws.com/debateinemo...
5,0.000568,0.004803,0.001290,9.495446e-06,0.189891,0.796831,0.001304,0.005304,True,380,0.056034,0.025914,0.003419,1.231909e-05,1.257774e-05,0.510783,0.403799,2.599900e-05,True,https://s3-us-west-2.amazonaws.com/debateinemo...
6,0.000099,0.002541,0.000583,1.592629e-05,0.001038,0.969495,0.009493,0.016734,True,385,0.091437,0.030152,0.004094,1.457128e-05,4.279568e-05,0.677865,0.196353,4.113510e-05,True,https://s3-us-west-2.amazonaws.com/debateinemo...
7,0.000989,0.002485,0.002013,7.679016e-05,0.027965,0.936948,0.004716,0.024807,True,390,0.363218,0.007548,0.001929,5.470980e-05,3.263133e-06,0.494955,0.132232,6.120257e-05,True,https://s3-us-west-2.amazonaws.com/debateinemo...
8,0.000625,0.001943,0.002891,3.523093e-05,0.017187,0.961799,0.009672,0.005846,True,395,0.172965,0.005057,0.076081,4.356739e-04,5.509598e-04,0.700848,0.007858,3.620347e-02,True,https://s3-us-west-2.amazonaws.com/debateinemo...
9,0.000052,0.001865,0.000440,1.633717e-05,0.006685,0.976300,0.007208,0.007433,True,400,0.090888,0.029770,0.008298,1.146629e-05,3.608993e-06,0.228934,0.642083,1.212075e-05,True,https://s3-us-west-2.amazonaws.com/debateinemo...


In [7]:
df.to_csv('debate3.csv', sep=',', index=False)